## Analysis sparks multiple images

In [ ]:
from os import listdir
from os.path import isfile, join
from sparks_image import *
from sparks_analysis import *
import cv2
import matplotlib.pyplot as plt
%matplotlib inline


path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/sp_para entrenar/'
files = [".".join(f.split(".")[:-1]) for f in listdir(path) if isfile(join(path, f))]
for photo_name in files:
    image = cv2.imread(path + photo_name + ".tif")    # Read image
    plt.imshow(image, cmap='gray')
    plt.show()
    cont = input('Continue?')
    if 's' in cont.lower():
        list_img = image_process (image,path,5)
        list_img_col = list_img [0]
        list_img_row = list_img [1]
        x = list_img[2]
        y = list_img[3]
        width = list_img[4]
        high = list_img[5]
        out_sparks = analysis_process (list_img_col, list_img_row,x,y,width,high)
        print (out_sparks)
#         pd.DataFrame.to_csv(out_sparks,path+photo_name)

## Sparks analysis single image

In [1]:
from sparks_image import *
from sparks_analysis import *
import cv2
from scipy import stats

# path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/Rata/C071112/'
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Imagenes_confocal/Rata/C071112/'
photo_name = 'c2ac009'
image = cv2.imread(path + photo_name + ".tif")    # Read image
list_img = image_process (image, 10)
if list_img:
    list_img_col = list_img [0]
    list_img_row = list_img [1]
    x = list_img[2]
    y = list_img[3]
    width = list_img[4]
    high = list_img[5]
    out_sparks = analysis_process (list_img_col, list_img_row,x,y,width,high)

In [2]:
out_sparks

,tiempo_maximo,intensidad_maxima,tiempo_minimo,intensidad_minima,tiempo_valle,intensidad_valle,amplitud,TTP,TTP50,FDHM,tau,(ΔF/F0)/ΔTmax,fullDuration,fullWidth,FWHM,pos_x,pos_y,width,high
0,3.0,80.750000,0,46.500000,4,69.500000,0.736559,3.0,1.824911,1.656352,6.665280,0.245520,4,9,4.687143,10,494,12,8
1,4.0,96.000000,2,65.000000,9,62.555556,0.476923,2.0,0.880222,4.095751,11.866685,0.238462,7,5,1.019839,66,465,9,11
2,5.0,116.470588,0,64.647059,11,82.117647,0.801638,5.0,2.728724,4.714344,17.534467,0.160328,11,12,5.987376,204,463,17,13
3,9.0,111.941176,2,51.764706,13,84.941176,1.162500,7.0,2.552472,5.953198,16.204371,0.166071,11,12,4.024433,79,454,17,16
4,4.0,117.066667,0,71.200000,8,95.200000,0.644195,4.0,2.930199,3.250083,19.950808,0.161049,8,5,2.699776,159,384,15,10
5,3.0,112.928571,1,65.285714,8,83.642857,0.729759,2.0,1.101477,3.309131,16.203999,0.364880,7,11,4.144902,176,311,14,11
6,9.0,122.500000,7,103.333333,10,93.611111,0.185484,2.0,1.073890,1.392591,3.717998,0.092742,3,9,3.645787,209,275,18,14
7,3.0,107.769231,0,69.153846,8,75.230769,0.558398,3.0,2.085171,3.833522,16.041097,0.186133,8,10,4.712187,141,82,13,11
8,10.0,170.473684,5,112.052632,27,94.421053,0.521372,5.0,2.751472,8.437447,33.486005,0.104274,22,4,1.099785,196,34,19,33
9,4.0,125.850000,0,75.050000,10,96.900000,0.676882,4.0,2.491352,4.307948,37.575089,0.169221,10,17,6.905454,183,13,20,12


## sparks_data

In [1]:
import prueba_image_spark

# path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/sp_para entrenar/'
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Imagenes_confocal/Rata/C071112/'
photo_name = 'c2ac009'

image = prueba_image_spark.Read_image(path, photo_name)
imcrop = prueba_image_spark.Image_cropping(image)
contours = prueba_image_spark.Image_processing(imcrop)
img_processed = prueba_image_spark.Image_processing(contours)
original = prueba_image_spark.Image_processing(img_processed)
img_data = prueba_image_spark.Image_analysis(contours, img_processed, original)

list_img_col = img_data[0]
list_img_row = img_data[1]
x_position = img_data[2]
y_position = img_data[3]
width = img_data[4]
high = img_data[5]
# Detect each event and give it a category
def write_points(event, x_position, y_position, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        img_points.append(flags)
    if event == cv2.EVENT_RBUTTONDOWN:
        img_points.append(flags)
winname="TAG :: Press ESC to exit; left Click to TAG 1; right Click to TAG 2"
cv2.namedWindow(winname)
img_points = []
cv2.setMouseCallback(winname,write_points)
while True:
    cv2.imshow(winname,img_processed)
    cv2.imshow('original',image)
    if cv2.waitKey(20) & 0xFF ==27:
        break      
cv2.destroyAllWindows()

# Take the processing data to analyze each spark and extract parameters

parameters = sparks_analysis.analysis_process (list_img_col, list_img_row,x,y,width,high,img_points)
print(parameters)
#parameters.to_csv()

TypeError: Expected Ptr<cv::UMat> for argument 'src'

In [ ]:
type(image)

from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

img_bc = interactive(f, brightness=100, contrast=100)   #display brightness and contrast
display (img_bc)

im = rotation (imCrop, 90)    # rotate image
display_image ('Image' , im)    # Display cropped image